# Chap 9. Backpropagation

## 1. Backpropagation Idea
<img src="https://devblogs.nvidia.com/wp-content/uploads/2015/08/training_inference1.png" alt="" title="" />

- Chap 8. 에서 언급한 XOR 문제를 logistic regression 3개와 forward propagation, back propagation 알고리즘을 이용하여 해결할수 있다.

- Forward propagation을 통과 시킨후 에러가 발생되면 backpropagation을 통하여 Weight과 Bias값 학습시키게 된다.

- 결국 Tensorflow는 모든 Tensor(노드를 이루는 Operation, 선을 이루는 Data)를 graph로 다루게되며, Gradient Descent Optimizer API를 통해 backward 방향으로 미분을 구하여 Backpropagation을 실현하게 되며 Machine learning을 이루어지게 한다.


## 2. Forward Propagation

- Logistic regression의 Hypothesis는 아래와 같았다.
  - logits = tf.matmul(X, W) + b
  - hypothesis = tf.sigmoid(logits)

- 하기 블록 다이어 그램에서 Neuron1, 2, 3은 각각의 logistic regression 이며 Hypothesis 설계에 필요한 값들은 아래와 같다.
  - input1은 x1, input2는 x2
  - W11, W12는 logistic regression1의 Weight
  - W21, W22는 logistic regression2의 Weight
  - W31, W32는 logistic regression3의 Weight
  - Bias1, 2, 3은 각 logistic regession의 bias 이다.
  - Neuron3의 입력은 기호가 표시되어 있지 않았으나 y1, y2라고 명한다.
  - Output은 Y hat으로서 XOR 진리표의 값과 확인할 것이다.


<img src="https://www.researchgate.net/profile/Medhat_Moussa/publication/228939274/figure/fig1/AS:393876184551431@1470918808455/Topology-of-ANN-used-to-solve-logic-XOR-problem.png" alt="" title="" />

- 진리표는 아래와 같다.

In [ ]:
'''
| x1 | x2 | y1 | y2 | y hat | XOR | 
|----|----|----|----|-------|-----| 
| 0  | 0  | 0  | 1  |   0   | 0   | 
| 0  | 1  | 0  | 0  |   1   | 1   | 
| 1  | 0  | 0  | 0  |   1   | 1   | 
| 1  | 1  | 1  | 0  |   0   | 0   | 
| 1  | 1  | 1  | 0  |   0   | 0   |
'''

- 진리표와 같은 결과가 나오는 과정은 아래와 같다.
 - x1과 x2는 진리표와 같이 각각 (0, 0), (0, 1), (1, 0), (1, 1)으로 주어 진다.
 - logistic regession1의 Weight은 (5, 5)이며  bias는 -8로 설정 하였다.
 - logistic regession2의 Weight은 (-7, -7) bias는 3으로 설정 하였다.
 - logistic regession3의 Weight은 (-11, -11) bias는 6으로 설정 하였다.

- Logistic regression1의 계산 과정은 아래와 같다. W = (5, 5),   b = -8 이다.
  - (0＊5 + 0＊5) + -8 =   0 - 8 = -8 이후 sigmoid(-8) = 0
  - (0＊5 + 1＊5) + -8 =   5 - 8 = -3 이후 sigmoid(-3) = 0
  - (1＊5 + 0＊5) + -8 =   5 - 8 = -3 이후 sigmoid(-3) = 0
  - (1＊5 + 1＊5) + -8 = 10 - 8 =   2 이후 sigmoid(  2) = 1

- Logistic regression1의 계산 과정은 아래와 같다. W = (-7, -7),   b = 3 이다.
  - (0＊-7 + 0＊-7) + 3 =   0 + 3 =  3  이후 sigmoid(3) = 1
  - (0＊-7 + 1＊-7) + 3 =  -7 + 3 = -4  이후 sigmoid(3) = 0
  - (1＊-7 + 0*＊7) + 3 =  -7 + 3 = -4  이후 sigmoid(3) = 0
  - (1＊-7 + 1＊-7) + 3 = -14 + 3 = -11 이후 sigmoid(3) = 0

- Logistic regression3의 계산 과정은 아래와 같다. W = (-11, -11),   b = 6 이다.
  - x1과 x2는 이전 결과의 조합이므로 (0, 1), (0, 0), (0, 0), (1, 0)으로 주어진다.
  - (0＊-11 + 1＊-11) + 6 = -11 + 6 = -5 이후 sigmoid(-5) = 0
  - (0＊-11 + 0＊-11) + 6 =   0 + 6 =  6 이후 sigmoid(  6) = 1
  - (0＊-11 + 0＊-11) + 6 =   0 + 6 =  6 이후 sigmoid(  6) = 1
  - (1＊-11 + 0＊-11) + 6 = -11 + 6 = -5 이후 sigmoid(-5) = 0
  
- 참고로 sigmoid function graph는 아래와 같다.
\begin{equation*}
W := W - \alpha \frac{\partial}{\partial W}cost(W)
\end{equation*}
<img src="https://ml4a.github.io/images/figures/sigmoid.png" alt="" title="" />

## 3. Neural Network
- 위 블록다이어그램에서 Logistic regression 1과 Logistic regression 2는 아래 그림의 Hidden 영역과 같이 1개의 Multinomial logistic regression 으로 만들수 있다.

<img src="https://i.stack.imgur.com/7g8Bj.png" alt="" title="" />


- Hidden 영역 Multinomial logistic regression의 Weight 값과 bias 값을 아래와 같이 2차원 벡터로 가지면 된다.

\begin{equation*}
W_1 = 
\begin{bmatrix}
5, -7 \\
5, -7 \\
\end{bmatrix}, b_1 =
\begin{bmatrix}
8, 3 \\
\end{bmatrix} \\
\end{equation*}

- Output 영역의 Logistic regression의 Weight 값과 bias 값은 아래와 같다.
\begin{equation*}
W_2 =
\begin{bmatrix}
-11, -11 \\
\end{bmatrix}, b_2 = 6
\end{equation*}

- Input을 X라 하고, Multinomial logistic regression의 hypothesis k(x)를 설계한다면 아래와 같다.
\begin{equation*}
K(x) = sigmoid(XW_1 + b_1)
\end{equation*}

- 최종적으로 Output 영역의 hypothesis H(x)를 설계한다면 아래와 같다.
\begin{equation*}
H(x) = sigmoid((K(x))W_2 + b_2)
\end{equation*}

- 최종적으로 설계된 hypothesis를 Tensorflow 코드로 구현하면 아래와 같다.
  - K = tf.sigmoid(tf.matmul(X, W1) + b1)
  - hypothesis = tf.sigmoid(tf.matmul(K, W2) + b2)

## 4. Backpropagation (How can we learn w1, w2, b1, b2 from traning data?)
- Forwardpropagation으로 traning data를 입력시키고, Backpropagation (derivative, chain rule)을 사용하여 학습을 시킬수 있다.

- 먼저 Forwardpropagation을 시킬때 아래와 같이 logits을 만들수 있다.
  - f = wx + b
  - wx 를 g라고 한다면 g = wx
  - f = g + b가 된다.
  - 위 식과 traning data를 입력하여 Tensor graph로 표현하면 아래와 같다.
    - g = w ＊ x = -2 ＊ 10 = -10
    - f = g + b = -7

In [ ]:
'''
w = -2
-------------\
∂f/∂w = 5     \
               \  g = -10   
x = 5          (*)------------\
---------------/ ∂f/∂g = 1     \ 
∂f/∂x = -2                      \
                                 \
b = 3                           (+)------- f = -7
--------------------------------/
∂f/∂b = 1
'''                  

- Backpropagation을 하는 목적은 w가 f에 미치는 영향, x가 f에 미치는 영향, b가 f에 미치는 영향을 알고 싶은 것이다. 이것을 알기 위해 역순으로 미분을 해나가는 것이다. 이 과정을 통하여 Weight과 Bias의 값을 조절하게 되며 결국 이것이 학습이 일어나는 것이다. neural network의 hidden layer가 아무리 많더라도 operation을 알기 때문에 역순으로 미분을 해나갈수 있다.


- 위 그래프에서 미분을 간단히 하기위해 아래 편미분을 먼저 수행한다.


- g = wx를 편미분하면 아래와 같다. (Chap 0의 2.2.1. 참조)

\begin{equation*}
\frac{\partial g}{\partial_ w} = x,
\end{equation*}

\begin{equation*}
\frac{\partial g}{\partial_ x}=w 
\end{equation*}

- f = g + b를 편미분하면 아래와 같다. (Chap 0의 2.2.2. 참조)

\begin{equation*}
\frac{\partial f}{\partial_ g} = 1,\\
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial_ b} = 1 \\
\end{equation*}

- Backpropagation을 위해 최종적으로 미분식을 정리하면 아래와 같다.

\begin{equation*}
\frac{\partial f}{\partial_ w} = \frac{\partial f}{\partial_ g} \text{＊} \frac{\partial g}{\partial_ w} = 1 \text{＊} x = 1 \text{＊} 5 = 5
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial_ x} = \frac{\partial f}{\partial_ g} \text{＊} \frac{\partial g}{\partial_ x} = 1 \text{＊} w = 1 \text{＊} -2 = -2 
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial_ b} = 1
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial_ g}  = 1
\end{equation*}

- 각 미분식의 의미는 아래와 같다.
  - ∂f/∂w = 5 는 w의 값이 최종 출력에 5배로 영향을 준다는 의미이다.
  - ∂f/∂x = -2 는 x의 값이 최종 출력에 -2배로 영향을 준다는 의미이다.
  - ∂f/∂b = 1 은 bias의 값이 최종 출력 f에 1:1로 비례한다는 의미이다.


- Sigmoid의 미분
  - sigmoid를 그래프로 펼치면 아래와 같고 역순으로 미분을 해 나간다.  

\begin{equation*}
g(z) = \frac{1}{(1+e^{-z})}
\end{equation*}

In [ ]:
'''
                                 x
z --- (* -1) --- (exp) --- (+1) --- (1/x) --- g
'''

- Neural network은 위와 같은 그래프들이 길게 늘어선것 이다. 끝부터 처음까지 역순으로 미분을 해 나가면서 backpropagation 구하고 학습을 가능하게 하는 것이다.

- Tensorflow가 모든 tensor를 graph로 다루는 이유도 미분을 하여 backpropagation을 구하기 위한것이다. 실제로 미분은 GradientDescentOptimizer 함수가 수행한다.

- Tensorflow에서는 이런 graph를 다루는 것을 시각화 하기 위해 Tensor Borad라는 매우 유용한 툴을 제공 한다.

<img src="https://cdn-images-1.medium.com/max/1600/1*aOYUa3hHKi9HlYnnFAipUQ.gif" alt="" title="" />

## 5. Lab1: Solving XOR problem with one logistic regression
- Logistic regression을 사용하는 이유는 XOR의 output이 0, 1과 같이 binary 형태이기 때문이다.
- 1개의 network(Logistic regression) 으로는 XOR 문제를 해결할 수 없음을 보여준다.

In [1]:
################################################################################
# lab9-1 : xor by logistic regression
################################################################################

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random

tf.set_random_seed(777) # for reproducibility
learning_rate = 0.1    

# XOR True Table input
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
# XOR True Table output
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, shape=[None, 2]) # XOR input X1, X2
Y = tf.placeholder(tf.float32, shape=[None, 1]) # XOR output Y

W = tf.Variable(tf.random_normal([2, 1], name='weight'))
b = tf.Variable(tf.random_normal([1], name='bias'))

# Hyphthesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

# Cost / loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
    tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize tensorflow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 5000 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}),
                sess.run(W))
    
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={
        X: x_data, Y: y_data})
    print("\nHypothesis:\n", h, "\nCorrect:\n", c, "\nAccuracy:\n", a)

0 0.87597954 [[0.7863567]
 [0.6628261]]
5000 0.6931472 [[1.3183484e-07]
 [1.3247467e-07]]
10000 0.6931472 [[1.3183484e-07]
 [1.3247467e-07]]

Hypothesis:
 [[0.5]
 [0.5]
 [0.5]
 [0.5]] 
Correct:
 [[0.]
 [0.]
 [0.]
 [0.]] 
Accuracy:
 0.5


## 6. Lab2: Solving XOR problem with neural network
- 2개의 Network(Logistic regression)을 이용하여 XOR 문제를 해결하고 있음을 보여준다.

- 위 설명에서는 XOR 문제를 해결하기 위해 3개의 Logistic regression을 이용한다고 설명하였다.  Neuron3의 입력을 Neuron1의 출력과 Neuron2의 출력에서 받았으나, 하기 Tensorflow 소스코드에서는 첫번째 Logistic regression의 W1, B1을 2개씩 출력하여 두번째 Logistic regssion의 입력으로 보내어 처리하였다. 즉 3개의 Logistic regression을 사용한것과 동일한 결과를 얻는다.

- 이와 같이 Tensorflow에서는 매우 쉽게 Weight과 Bias의 출력을 wide하고, Network의 깊이를 deep하게 처리할수 있다.

In [2]:
################################################################################
# lab9-2 : xor by neural nerwork
################################################################################

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random

tf.set_random_seed(777) # for reproducibility
learning_rate = 0.1    

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

'''
W = tf.Variable(tf.random_normal([2, 1], name='weight'))
b = tf.Variable(tf.random_normal([1], name='bias'))

# Hyphthesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
'''
W1 = tf.Variable(tf.random_normal([2, 2], name='weight1'))
b1 = tf.Variable(tf.random_normal([2], name='bias1'))
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([2, 1], name='weight2'))
b2 = tf.Variable(tf.random_normal([1], name='bias2'))
hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)


# Cost / loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
    tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize tensorflow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 5000 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}),
                sess.run([W1, W2]))
    
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={
        X: x_data, Y: y_data})
    print("\nHypothesis:\n", h, "\nCorrect:\n", c, "\nAccuracy:\n", a)

0 1.2737582 [array([[ 2.1355143 , -0.0782264 ],
       [ 0.04496152, -0.48487958]], dtype=float32), array([[-0.6957488],
       [-1.3244522]], dtype=float32)]
5000 0.07993939 [array([[ 3.6978915, -5.4047523],
       [ 3.6858444, -5.338445 ]], dtype=float32), array([[-6.841885 ],
       [-7.2329164]], dtype=float32)]
10000 0.016905494 [array([[ 4.8416247, -6.316066 ],
       [ 4.8371215, -6.2909093]], dtype=float32), array([[-9.921189],
       [-9.878948]], dtype=float32)]

Hypothesis:
 [[0.01368036]
 [0.98169047]
 [0.9817343 ]
 [0.01679078]] 
Correct:
 [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
 1.0


## 7. Lab3: Solving XOR problem with wide and deep neural network
- Lab2에 비하여 학습이 잘되는 것을 확인할 수 있다.

In [ ]:
'''
# Lab2 Result
Hypothesis:
 [[0.01368036]
 [0.98169047]
 [0.9817343 ]
 [0.01679078]]
 
# Lab3 Result
Hypothesis:
 [[9.6710393e-04]
 [9.9877375e-01]
 [9.9876839e-01]
 [1.8678077e-03]]  
'''

In [3]:
################################################################################
# lab9-3 : xor by neural nerwork wide deep
################################################################################

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random

tf.set_random_seed(777) # for reproducibility
learning_rate = 0.1    

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

'''
W = tf.Variable(tf.random_normal([2, 1], name='weight'))
b = tf.Variable(tf.random_normal([1], name='bias'))

# Hyphthesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
'''
'''
W1 = tf.Variable(tf.random_normal([2, 2], name='weight1'))
b1 = tf.Variable(tf.random_normal([2], name='bias1'))
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([2, 1], name='weight2'))
b2 = tf.Variable(tf.random_normal([1], name='bias2'))
hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)
'''
W1 = tf.Variable(tf.random_normal([2, 10], name='weight1'))
b1 = tf.Variable(tf.random_normal([10], name='bias1'))
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([10, 10], name='weight2'))
b2 = tf.Variable(tf.random_normal([10], name='bias2'))
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([10, 10], name='weight1'))
b3 = tf.Variable(tf.random_normal([10], name='bias1'))
layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)

W4 = tf.Variable(tf.random_normal([10, 1], name='weight2'))
b4 = tf.Variable(tf.random_normal([1], name='bias2'))
hypothesis = tf.sigmoid(tf.matmul(layer3, W4) + b4)

# Cost / loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
    tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize tensorflow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 5000 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}),
                sess.run([W1, W2, W3, W4]))
    
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={
        X: x_data, Y: y_data})
    print("\nHypothesis:\n", h, "\nCorrect:\n", c, "\nAccuracy:\n", a)

0 1.5568961 [array([[-0.60164016,  0.8919202 ,  0.35191226,  0.56761146, -0.99739665,
         0.32676136, -0.0275667 , -0.7428496 , -0.95138216, -2.952086  ],
       [ 2.1419542 ,  1.1751897 ,  0.3820754 , -1.32208   , -0.01911528,
         0.16187686, -3.0404184 ,  0.2068328 ,  0.3681465 , -1.5922024 ]],
      dtype=float32), array([[-0.8270249 , -0.5908381 , -0.9152915 , -1.6387377 , -0.39875317,
        -0.07010718, -0.62081033, -1.494441  ,  0.23517573, -0.9144238 ],
       [-0.7995736 ,  1.6431977 ,  0.7156355 ,  1.8664869 , -0.38515177,
         0.40816587, -0.6172208 , -0.09925716,  0.2734332 , -1.2857459 ],
       [-0.08891804,  0.20978071, -0.9512092 ,  0.33139288, -2.0785854 ,
        -0.48373953,  1.2817999 ,  0.48855504,  0.47712448,  0.98934746],
       [ 1.07698   ,  1.2265451 , -0.7964994 , -0.08373559, -0.36422762,
        -0.32580382, -0.06039998,  2.4420319 ,  0.18790258,  0.5140296 ],
       [-1.050017  ,  1.1016577 , -0.998483  , -0.6537007 , -0.05316174,
        -

## 8. Lab4: Solving XOR problem with backpropagation
- 이 예제는 Lab2: Solving XOR problem with neural network 예제와 동일한 결과를 갖지만  미분을 통해 backpropagation을 직접 구현하였다.

- Lab2는 train = tf.train.GradientDescentOptimizer() 함수를 이용하여 backpropagation을 간단히 구현한 것이다.

In [2]:
################################################################################
# lab9-8 : xor neural network back propagation
#
################################################################################

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random

tf.set_random_seed(777) # for reproducibility

learning_rate = 0.1

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
b1 = tf.Variable(tf.random_normal([2]), name='bias1')
l1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
b2 = tf.Variable(tf.random_normal([1]), name='bias2')
Y_pred = tf.sigmoid(tf.matmul(l1, W2) + b2)

# cost/loss function
# ref: see lecture 5-2 cost function of logistic regression    
cost = -tf.reduce_mean(Y * tf.log(Y_pred) + (1 - Y) *
                       tf.log(1 - Y_pred))

# Network
#          p1     a1           l1     p2     a2           l2 (y_pred)
# X -> (*) -> (+) -> (sigmoid) -> (*) -> (+) -> (sigmoid) -> (loss)
#       ^      ^                   ^      ^
#       |      |                   |      |
#       W1     b1                  W2     b2

# Loss derivative
d_Y_pred = (Y_pred - Y) / (Y_pred * (1.0 - Y_pred) + 1e-7)

# Layer 2
d_sigma2 = Y_pred * (1 - Y_pred)
d_a2 = d_Y_pred * d_sigma2
d_p2 = d_a2
d_b2 = d_a2
d_W2 = tf.matmul(tf.transpose(l1), d_p2)

# Mean
d_b2_mean = tf.reduce_mean(d_b2, axis=[0])
d_W2_mean = d_W2 / tf.cast(tf.shape(l1)[0], dtype=tf.float32)

# Layer 1
d_l1 = tf.matmul(d_p2, tf.transpose(W2))
d_sigma1 = l1 * (1 - l1)
d_a1 = d_l1 * d_sigma1
d_b1 = d_a1
d_p1 = d_a1
d_W1 = tf.matmul(tf.transpose(X), d_a1)

# Mean
d_W1_mean = d_W1 / tf.cast(tf.shape(X)[0], dtype=tf.float32)
d_b1_mean = tf.reduce_mean(d_b1, axis=[0])

# Weight update
step = [
  tf.assign(W2, W2 - learning_rate * d_W2_mean),
  tf.assign(b2, b2 - learning_rate * d_b2_mean),
  tf.assign(W1, W1 - learning_rate * d_W1_mean),
  tf.assign(b1, b1 - learning_rate * d_b1_mean)
]

# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(Y_pred > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    print("shape", sess.run(tf.shape(X)[0], feed_dict={X: x_data}))


    for i in range(10001):
        sess.run([step, cost], feed_dict={X: x_data, Y: y_data})
        if i % 5000 == 0:
            print(i, sess.run([cost, d_W1], feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))

    # Accuracy report
    h, c, a = sess.run([Y_pred, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis:\n", h, "\nCorrect:\n", c, "\nAccuracy:\n", a)

shape 4
0 [1.047763, array([[0.1790797 , 0.03775062],
       [0.15853871, 0.03622694]], dtype=float32)] [array([[-0.22568084, -0.53591216],
       [ 0.16982368, -0.41798908]], dtype=float32), array([[-1.0534035],
       [-0.2579086]], dtype=float32)]
5000 [0.5206581, array([[0.03393272, 0.05396489],
       [0.05865538, 0.03833643]], dtype=float32)] [array([[-2.0188885 , -3.525843  ],
       [-0.02829779, -2.9756508 ]], dtype=float32), array([[-2.5196912],
       [ 3.7735164]], dtype=float32)]
10000 [0.024093203, array([[0.00685279, 0.00462249],
       [0.00716581, 0.00493447]], dtype=float32)] [array([[-5.5450306, -5.335227 ],
       [-5.4966826, -5.3049097]], dtype=float32), array([[-9.491918],
       [ 8.734669]], dtype=float32)]

Hypothesis:
 [[0.0209769 ]
 [0.97834617]
 [0.97833496]
 [0.03089069]] 
Correct:
 [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
 1.0


## 9. Lab5: Tensor Board with solving XOR problem example
- step 1: From TF graph, decide which tensors you want to log
  - 로깅할 값이 한개면 scalar, 여러개면 histogram을 사용한다.
  - w1_hist = tf.summary.histogram("weights1", W1)
  - cost_summ = tf.summary.scalar("cost", cost)


- step 2: Merge all summaries
  - merged_summary = tf.summary.merge_all()
  
  
- step 3: Create writer and add graph
  - writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
  - writer.add_graph(sess.graph) # show the graph


- step 4: Run summary merge and add_summary
  - summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
  - writer.add_summary(summary, global_step=step)


- step 5: Launch the Tensorboard
  - $ tensorboard --logdir=./logs/xor_logs_r0_01


- step 6: Launch browser with localhost
  - http://127.0.0.1:6006
  - ssh의 포트 포워딩을 이용하여 리모트 서버에 접속하여 볼수도 있다.
    - $ ssh -L local_port:127.0.0.1:remote_port username@server.com
      - local> ssh -L 7007:127.0.0.1:6006 xxx@server.com
      - server> tensorborad --logdir=./logs/xor_logs_r0_01


- Tensor Board에서 그래프를 볼때 계층을 구분해서 보고 싶을 때는 name_scope을 사용한다.
    - with tf.name_scope("layer1"):


- step3에서 parent 로그 디렉토리("logs/") 하위에 여러개의 child 디렉토리를 만들어 그래프를 저장하면 tensor board에서 그래프를 비교 분석할수 있다. 단, tensorboard 실행시 parent 로그 디렉토리를 지정해 줘야 한다.
  - writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
  - writer = tf.summary.FileWriter("./logs/xor_logs_r0_02")
  - $ tensorboard --logdir=./logs

In [1]:
################################################################################
# lab9-4 : Tensorboard with xor
################################################################################

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random

tf.set_random_seed(777) # for reproducibility
learning_rate = 0.1    

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, shape=[None, 2], name='x-input')
Y = tf.placeholder(tf.float32, shape=[None, 1], name='y-input')

with tf.name_scope("layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2], name='weight1'))
    b1 = tf.Variable(tf.random_normal([2], name='bias1'))
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("baises1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)

with tf.name_scope("layer2"):    
    W2 = tf.Variable(tf.random_normal([2, 1], name='weight2'))
    b2 = tf.Variable(tf.random_normal([1], name='bias2'))
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("baises2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

# Cost / loss function
with tf.name_scope("cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
        tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train"):
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

# Launch graph
with tf.Session() as sess:
    # Tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph) # show the graph

    # Initialize tensorflow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={
            X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

        if step % 5000 == 0:
             print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}),
                sess.run([W1, W2]))
    
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={
        X: x_data, Y: y_data})
    print("\nHypothesis:\n", h, "\nCorrect:\n", c, "\nAccuracy:\n", a)

0 0.71064335 [array([[ 0.70269835,  0.77858704],
       [-1.117285  , -0.20518988]], dtype=float32), array([[1.8076982 ],
       [0.44572306]], dtype=float32)]
5000 2.5004463e-05 [array([[ -9.919293,  10.363377],
       [  9.945174, -10.647296]], dtype=float32), array([[21.51126],
       [21.62779]], dtype=float32)]
10000 1.7881409e-06 [array([[-10.754227 ,  11.173415 ],
       [ 10.777167 , -11.4593525]], dtype=float32), array([[26.690191],
       [26.812712]], dtype=float32)]

Hypothesis:
 [[1.9362387e-06]
 [9.9999821e-01]
 [9.9999845e-01]
 [1.9069422e-06]] 
Correct:
 [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
 1.0
